# Gradio MCP Playground - Manual Testing Notebook

This notebook provides a comprehensive manual testing environment for the Gradio MCP Playground.
Use this to validate functionality, debug issues, and explore features interactively.

## 📋 Prerequisites

1. Install the package: `pip install -e .`
2. Ensure all dependencies are installed
3. Run this notebook in the project root directory

## 🛠️ Setup and Imports

✅ **Good News**: The project now gracefully handles missing dependencies!

### 🎯 **Testing Levels Available:**

1. **🔧 Core Testing** (requires: click, pydantic, rich)
   - ✅ CLI functionality testing  
   - ✅ Configuration management
   - ✅ Server registry operations

2. **🌐 Web Testing** (requires: gradio)
   - ✅ Dashboard creation
   - ✅ Web UI functionality

3. **🚀 Full MCP Testing** (requires: mcp package)
   - ✅ MCP server creation
   - ✅ Tool registration and execution
   - ✅ Client connections

### 🔧 **Quick Installation:**

Choose your level of functionality:

```bash
# For basic CLI testing only
pip install --user click pydantic rich

# For web dashboard too  
pip install --user gradio click pydantic rich

# For full MCP functionality
pip install --user gradio mcp click pydantic rich

# Or install everything (recommended)
pip install --user -e .
```

The notebook will automatically detect what's available and run appropriate tests!

In [5]:
# CRITICAL: Install the package in the current Python environment
import subprocess
import sys
import os
from pathlib import Path

print("🚀 Gradio MCP Playground - Installation Fix")
print("=" * 50)
print(f"🐍 Python: {sys.executable}")
print(f"📁 Working dir: {Path.cwd()}")

def install_and_test():
    """Install package and test CLI functionality"""
    
    # Step 1: Install the project in development mode
    print("\n🔧 Installing project in development mode...")
    install_commands = [
        [sys.executable, "-m", "pip", "install", "--user", "-e", ".."],
        [sys.executable, "-m", "pip", "install", "--user", "--break-system-packages", "-e", ".."],
    ]
    
    installed = False
    for cmd in install_commands:
        try:
            print(f"   Trying: {' '.join(cmd)}")
            subprocess.check_call(cmd)
            print(f"   ✅ Installation successful")
            installed = True
            break
        except subprocess.CalledProcessError as e:
            print(f"   ❌ Failed: {e}")
            continue
    
    if not installed:
        print("❌ Could not install package")
        return False
    
    # Step 2: Test CLI functionality
    print("\n🧪 Testing CLI functionality...")
    try:
        result = subprocess.run([
            sys.executable, "-m", "gradio_mcp_playground.cli", "--help"
        ], capture_output=True, text=True, timeout=10)
        
        if result.returncode == 0:
            print("✅ CLI working correctly!")
            return True
        else:
            print(f"❌ CLI error: {result.stderr}")
            return False
    except Exception as e:
        print(f"❌ CLI test failed: {e}")
        return False

# Run the installation and test
success = install_and_test()

if success:
    print("\n🎉 SUCCESS! Package is now properly installed!")
    print("✅ You can proceed with the rest of the notebook")
else:
    print("\n❌ INSTALLATION FAILED")
    print("💡 Try the following:")
    print("   1. Restart the Jupyter kernel")
    print("   2. Run this cell again")
    print("   3. Or run manually: pip install --user -e .")

print("\n" + "=" * 50)

🚀 Gradio MCP Playground - Installation Fix
🐍 Python: c:\Python313\python.exe
📁 Working dir: c:\Users\seanp\gradio-mcp-playground\tests

🔧 Installing project in development mode...
   Trying: c:\Python313\python.exe -m pip install --user -e ..
   ✅ Installation successful

🧪 Testing CLI functionality...
✅ CLI working correctly!

🎉 SUCCESS! Package is now properly installed!
✅ You can proceed with the rest of the notebook



In [6]:
import sys
import os
import json
import subprocess
import time
import asyncio
from pathlib import Path
import tempfile
import shutil

# Add project to path
sys.path.insert(0, str(Path.cwd()))

print(f"📁 Working directory: {Path.cwd()}")
print(f"🐍 Python executable: {sys.executable}")
print(f"🐍 Python version: {sys.version}")

# Test CLI installation first
print("\n🔍 Testing CLI installation...")
try:
    result = subprocess.run([sys.executable, "-m", "gradio_mcp_playground.cli", "--help"], 
                          capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("✅ CLI module is properly installed and accessible")
        cli_available = True
    else:
        print(f"❌ CLI module error: {result.stderr}")
        cli_available = False
except subprocess.TimeoutExpired:
    print("❌ CLI command timed out")
    cli_available = False
except Exception as e:
    print(f"❌ CLI test failed: {e}")
    cli_available = False

# Test gmp alias if available
print("\n🔍 Testing gmp alias...")
try:
    result = subprocess.run(["gmp", "--help"], capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("✅ gmp alias works")
        gmp_available = True
    else:
        print("⚠️ gmp alias not available (this is normal on some systems)")
        gmp_available = False
except:
    print("⚠️ gmp alias not available (this is normal on some systems)")
    gmp_available = False

# Check dependency availability
dependencies_status = {}

# Check core dependencies
core_deps = ["gradio", "click", "rich", "pydantic", "aiohttp"]
for dep in core_deps:
    try:
        __import__(dep)
        dependencies_status[dep] = True
        print(f"✅ {dep} available")
    except ImportError:
        dependencies_status[dep] = False
        print(f"❌ {dep} missing")

# Check optional dependencies
optional_deps = ["mcp", "anthropic", "huggingface_hub", "psutil"]
for dep in optional_deps:
    try:
        __import__(dep)
        dependencies_status[dep] = True
        print(f"✅ {dep} (optional) available")
    except ImportError:
        dependencies_status[dep] = False
        print(f"⚠️ {dep} (optional) missing - some features will be limited")

# Try to import project modules with dependency checks
basic_modules_available = False
if cli_available:
    try:
        from gradio_mcp_playground.config_manager import ConfigManager
        from gradio_mcp_playground.registry import ServerRegistry
        print("✅ Basic project modules imported successfully")
        basic_modules_available = True
    except ImportError as e:
        print(f"❌ Basic modules import error: {e}")

# Try importing MCP modules (these now handle missing dependencies gracefully)
mcp_modules_available = False
if dependencies_status.get("mcp", False):
    try:
        from gradio_mcp_playground.server_manager import MCPServer, GradioMCPServer
        from gradio_mcp_playground.client_manager import GradioMCPClient
        print("✅ MCP modules imported successfully")
        mcp_modules_available = True
    except ImportError as e:
        print(f"⚠️ MCP modules import error (expected if mcp not installed): {e}")

# Try importing web UI modules (now handles missing gradio gracefully)
web_modules_available = False
if dependencies_status.get("gradio", False):
    try:
        from gradio_mcp_playground.web_ui import create_dashboard
        print("✅ Web UI modules imported successfully")
        web_modules_available = True
    except ImportError as e:
        print(f"⚠️ Web UI modules import error: {e}")

# Summary
print(f"\n🔧 Module Availability Summary:")
print(f"  CLI: {'✅' if cli_available else '❌'}")
print(f"  Basic modules: {'✅' if basic_modules_available else '❌'}")
print(f"  MCP modules: {'✅' if mcp_modules_available else '⚠️ (requires mcp package)'}")
print(f"  Web UI: {'✅' if web_modules_available else '⚠️ (requires gradio)'}")

# Set global flags for test conditions
all_imports_successful = mcp_modules_available and web_modules_available and cli_available
core_imports_successful = cli_available and basic_modules_available

if all_imports_successful:
    print("🎉 All modules loaded - full testing available!")
elif core_imports_successful:
    print("⚡ Core modules loaded - basic testing available!")
else:
    print("⚠️ Limited functionality - some modules not accessible")
    print("💡 Make sure you ran the installation cell above successfully")
    print("💡 Try restarting the kernel and running all cells again")

📁 Working directory: c:\Users\seanp\gradio-mcp-playground\tests
🐍 Python executable: c:\Python313\python.exe
🐍 Python version: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]

🔍 Testing CLI installation...
✅ CLI module is properly installed and accessible

🔍 Testing gmp alias...
✅ gmp alias works
✅ gradio available
✅ click available
✅ rich available
✅ pydantic available
✅ aiohttp available
✅ mcp (optional) available
✅ anthropic (optional) available
✅ huggingface_hub (optional) available
✅ psutil (optional) available
✅ Basic project modules imported successfully
✅ MCP modules imported successfully
✅ Web UI modules imported successfully

🔧 Module Availability Summary:
  CLI: ✅
  Basic modules: ✅
  MCP modules: ✅
  Web UI: ✅
🎉 All modules loaded - full testing available!


## 🔧 Test Environment Setup

In [7]:
# Create a temporary test directory
test_dir = Path("test_environment")
test_dir.mkdir(exist_ok=True)

print(f"🏗️ Test environment: {test_dir.absolute()}")

# Helper function to run CLI commands with better error handling
def run_cli_command(command_list, capture_output=True, timeout=30):
    """Run a CLI command and return the result"""
    try:
        result = subprocess.run(
            [sys.executable, "-m", "gradio_mcp_playground.cli"] + command_list,
            capture_output=capture_output,
            text=True,
            timeout=timeout,
            encoding='utf-8',  # Explicit encoding to avoid Windows issues
            errors='replace'   # Handle encoding errors gracefully
        )
        return {
            "success": result.returncode == 0,
            "returncode": result.returncode,
            "stdout": result.stdout if capture_output else "",
            "stderr": result.stderr if capture_output else ""
        }
    except subprocess.TimeoutExpired:
        return {"success": False, "error": "Command timed out"}
    except Exception as e:
        return {"success": False, "error": str(e)}

print("✅ Helper functions defined")

🏗️ Test environment: c:\Users\seanp\gradio-mcp-playground\tests\test_environment
✅ Helper functions defined


## 📋 Basic CLI Testing

In [8]:
# Test basic CLI functionality
print("🧪 Testing basic CLI commands...")

# Test help command
result = run_cli_command(["--help"])
print(f"Help command: {'✅' if result['success'] else '❌'}")
if not result['success']:
    print(f"❌ Error: {result.get('stderr', result.get('error', 'Unknown error'))}")

# Test version command  
result = run_cli_command(["--version"])
print(f"Version command: {'✅' if result['success'] else '❌'}")
if result['success']:
    print(f"📦 Version: {result['stdout'].strip()}")

# Test subcommand help
subcommands = ["server", "client", "registry"]
for subcmd in subcommands:
    result = run_cli_command([subcmd, "--help"])
    print(f"{subcmd} help: {'✅' if result['success'] else '❌'}")
    if not result['success']:
        print(f"  Error: {result.get('stderr', 'Unknown error')}")

print("✅ Basic CLI tests completed")

🧪 Testing basic CLI commands...
Help command: ✅
Version command: ✅
📦 Version: gmp, version 0.1.0
server help: ✅
client help: ✅
registry help: ✅
✅ Basic CLI tests completed


## 🖥️ Server Management Testing

In [9]:
# Change to test directory for server creation
original_cwd = os.getcwd()
os.chdir(test_dir)

print(f"📁 Working in: {Path.cwd()}")
print(f"📝 Directory writable: {os.access('.', os.W_OK)}")

# Test server creation with basic template
server_name = "test-basic-server"
print(f"🧪 Creating server '{server_name}' with basic template...")

result = run_cli_command(["server", "create", server_name, "--template", "basic"])

server_created = result['success']
print(f"Server creation: {'✅' if server_created else '❌'}")

if server_created:
    print("📦 Server created successfully!")
    # Check if files were actually created
    server_path = Path(server_name)
    if server_path.exists():
        print(f"📁 Server directory: {server_path}")
        files = list(server_path.glob("*"))
        print(f"📄 Files created: {[f.name for f in files]}")
    else:
        print("⚠️ Server directory not found despite success")
        server_created = False
else:
    print("❌ Server creation failed")
    print(f"   Command: python -m gradio_mcp_playground.cli server create {server_name} --template basic")
    print(f"   Working dir: {Path.cwd()}")
    print(f"   Return code: {result.get('returncode', 'N/A')}")
    if result.get('stdout'):
        print(f"   Stdout: {result['stdout'][:200]}")
    if result.get('stderr'):
        print(f"   Stderr: {result['stderr'][:200]}")
    if result.get('error'):
        print(f"   Exception: {result['error']}")

# Store server creation status for next cell
globals()['server_creation_successful'] = server_created

📁 Working in: c:\Users\seanp\gradio-mcp-playground\tests\test_environment
📝 Directory writable: True
🧪 Creating server 'test-basic-server' with basic template...
Server creation: ✅
📦 Server created successfully!
📁 Server directory: test-basic-server
📄 Files created: ['app.py', 'README.md', 'requirements.txt']


In [10]:
# Test server listing
print("🧪 Testing server listing...")

result = run_cli_command(["server", "list"])
print(f"Server list: {'✅' if result['success'] else '❌'}")

if result['success']:
    if result['stdout'].strip() and result['stdout'].strip() != 'None':
        print("📋 Found servers in the list")
    else:
        print("📋 No servers currently listed (this is normal)")
else:
    print(f"❌ Error: {result.get('stderr', 'Unknown error')}")

🧪 Testing server listing...
Server list: ✅
📋 Found servers in the list


In [11]:
# Test server info (only if server was created successfully)
print("🧪 Testing server info...")

# Check if server was created in the previous cell
server_created = globals().get('server_creation_successful', False)

if server_created and Path(server_name).exists():
    result = run_cli_command(["server", "info", server_name])
    print(f"Server info: {'✅' if result['success'] else '❌'}")
    if result['success']:
        # Check if we got actual server info or "not found" message
        if 'not found' in result['stdout'].lower():
            print("⚠️ Server exists as directory but not registered in system")
        else:
            print("ℹ️ Server information retrieved successfully")
    else:
        print(f"❌ Error: {result.get('stderr', 'Unknown error')}")
else:
    print("⏭️ Skipping server info test (server creation failed)")
    # Test the command anyway to verify it handles missing servers correctly
    result = run_cli_command(["server", "info", "nonexistent-server"])
    if result['success'] and 'not found' in result['stdout'].lower():
        print("✅ Correctly handled request for nonexistent server")
    else:
        print("⚠️ Unexpected behavior for nonexistent server")

🧪 Testing server info...
Server info: ✅
⚠️ Server exists as directory but not registered in system


## 🔌 MCP Server Functionality Testing

In [12]:
# Test MCP server creation and tool registration
print("Testing MCP server functionality...")

# Check if MCP modules are available
if all_imports_successful and 'MCPServer' in globals():
    # Create a test MCP server
    mcp_server = MCPServer("test-mcp-server", "1.0.0", "Test MCP Server")

    @mcp_server.tool("greet", "Greet someone by name")
    async def greet(name: str) -> str:
        return f"Hello, {name}! Nice to meet you."

    @mcp_server.tool("calculate", "Perform basic arithmetic")
    async def calculate(a: float, b: float, operation: str) -> float:
        if operation == "add":
            return a + b
        elif operation == "subtract":
            return a - b
        elif operation == "multiply":
            return a * b
        elif operation == "divide":
            return a / b if b != 0 else float('inf')
        else:
            return 0.0

    @mcp_server.tool("process_text", "Process text in various ways")
    async def process_text(text: str, operation: str) -> str:
        if operation == "upper":
            return text.upper()
        elif operation == "lower":
            return text.lower()
        elif operation == "reverse":
            return text[::-1]
        elif operation == "length":
            return f"Length: {len(text)}"
        else:
            return text

    print(f"✅ MCP server created with {len(mcp_server.tools)} tools")
    for tool in mcp_server.tools:
        print(f"  🔧 {tool.name}: {tool.description}")
else:
    print("⏭️ Skipping MCP server tests - dependencies not available")
    print("💡 Install missing dependencies and restart the notebook")

Testing MCP server functionality...
✅ MCP server created with 3 tools
  🔧 greet: Greet someone by name
  🔧 calculate: Perform basic arithmetic
  🔧 process_text: Process text in various ways


In [13]:
# Test tool execution
print("Testing tool execution...")

if all_imports_successful and 'mcp_server' in globals():
    async def test_tools():
        # Test greet tool
        greet_tool = next(tool for tool in mcp_server.tools if tool.name == "greet")
        result = await greet_tool.handler("World")
        print(f"Greet tool result: {result}")
        
        # Test calculate tool
        calc_tool = next(tool for tool in mcp_server.tools if tool.name == "calculate")
        result = await calc_tool.handler(10.0, 5.0, "add")
        print(f"Calculate tool result: {result}")
        
        # Test process_text tool
        text_tool = next(tool for tool in mcp_server.tools if tool.name == "process_text")
        result = await text_tool.handler("Hello World", "upper")
        print(f"Process text tool result: {result}")

    await test_tools()
    print("✅ Tool execution tests completed")
else:
    print("⏭️ Skipping tool execution tests - MCP server not available")

Testing tool execution...
Greet tool result: Hello, World! Nice to meet you.
Calculate tool result: 15.0
Process text tool result: HELLO WORLD
✅ Tool execution tests completed


In [14]:
# Test Gradio integration
print("Testing Gradio integration...")

if all_imports_successful and 'mcp_server' in globals():
    try:
        gradio_functions = mcp_server.to_gradio_functions()
        print(f"✅ Converted {len(gradio_functions)} tools to Gradio functions")
        
        for func in gradio_functions:
            print(f"  📋 {func['name']}: {func['description']}")
            print(f"    Inputs: {len(func['inputs'])} components")
            print(f"    Output: {func['outputs']}")
            
    except Exception as e:
        print(f"❌ Gradio integration error: {e}")
else:
    print("⏭️ Skipping Gradio integration tests - dependencies not available")

Testing Gradio integration...
✅ Converted 3 tools to Gradio functions
  📋 greet: Greet someone by name
    Inputs: 1 components
    Output: text
  📋 calculate: Perform basic arithmetic
    Inputs: 3 components
    Output: text
  📋 process_text: Process text in various ways
    Inputs: 2 components
    Output: text


## 🌐 Web Dashboard Testing

In [15]:
# Test dashboard creation
print("🧪 Testing web dashboard creation...")

if all_imports_successful and 'create_dashboard' in globals():
    try:
        # Suppress Gradio warnings during dashboard creation
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            dashboard = create_dashboard()
        
        print("✅ Dashboard created successfully")
        print(f"📊 Dashboard type: {type(dashboard).__name__}")
        print("💡 To test interactively: gmp dashboard")
        
    except Exception as e:
        print(f"❌ Dashboard creation error: {e}")
else:
    print("⏭️ Skipping dashboard tests - web UI dependencies not available")

🧪 Testing web dashboard creation...
✅ Dashboard created successfully
📊 Dashboard type: Blocks
💡 To test interactively: gmp dashboard


## 📦 Registry Testing

In [16]:
# Test registry functionality
print("Testing registry functionality...")

if 'ServerRegistry' in globals():
    try:
        registry = ServerRegistry()
        print("✅ Registry created successfully")
        
        # Test template listing
        templates = registry.list_templates()
        print(f"📚 Available templates: {templates}")
        
        # Test categories
        categories = registry.list_categories()
        print(f"🏷️ Available categories: {categories}")
        
        # Test template existence check
        for template in ["basic", "calculator", "nonexistent"]:
            exists = registry.template_exists(template)
            print(f"  Template '{template}': {'✅' if exists else '❌'}")
            
    except Exception as e:
        print(f"❌ Registry error: {e}")
else:
    print("⏭️ Skipping registry tests - ServerRegistry not available")

Testing registry functionality...
✅ Registry created successfully
📚 Available templates: ['basic', 'calculator', 'basic-tool', 'image-generator', 'multi-tool']
🏷️ Available categories: ['advanced', 'ai', 'data', 'integration', 'starter', 'tools', 'web']
  Template 'basic': ✅
  Template 'calculator': ✅
  Template 'nonexistent': ❌


In [17]:
# Test registry CLI commands  
print("🧪 Testing registry CLI commands...")

# Test categories
result = run_cli_command(["registry", "categories"])
print(f"Registry categories: {'✅' if result['success'] else '❌'}")
if result['success'] and result['stdout'].strip():
    # Only show the key info, not full output
    lines = result['stdout'].strip().split('\n')
    category_lines = [line for line in lines if ' - ' in line and '(' in line]
    print(f"📚 Found {len(category_lines)} categories")
else:
    print(f"❌ Error: {result.get('stderr', 'Unknown error')}")

# Test search
result = run_cli_command(["registry", "search"])
print(f"Registry search: {'✅' if result['success'] else '❌'}")
if result['success'] and 'Found' in result['stdout']:
    # Extract just the count
    first_line = result['stdout'].split('\n')[0]
    print(f"🔍 {first_line}")
else:
    print(f"❌ Error: {result.get('stderr', 'Unknown error')}")

print("✅ Registry CLI tests completed")

🧪 Testing registry CLI commands...


[06/03/25 09:14:12] INFO     HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK" ]8;id=746642;file://C:\Users\seanp\AppData\Roaming\Python\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=314744;file://C:\Users\seanp\AppData\Roaming\Python\Python313\site-packages\httpx\_client.py#1025\1025]8;;\

Registry categories: ✅
📚 Found 7 categories
Registry search: ✅
🔍 Found 10 server(s):
✅ Registry CLI tests completed


## 🔧 Configuration Management Testing

In [18]:
# Test configuration management
print("Testing configuration management...")

if 'ConfigManager' in globals():
    try:
        config_manager = ConfigManager()
        print("✅ Config manager created")
        
        # Test default configuration
        default_config = {
            "default_port": 7860,
            "auto_reload": True,
            "mcp_protocol": "auto",
            "log_level": "INFO"
        }
        
        # Save test configuration
        config_manager.save_config(default_config)
        print("✅ Configuration saved")
        
        # Load configuration
        loaded_config = config_manager.load_config()
        print(f"✅ Configuration loaded: {loaded_config}")
        
        # Test individual settings
        print(f"Default port: {config_manager.default_port}")
        print(f"Auto reload: {config_manager.auto_reload}")
        print(f"MCP protocol: {config_manager.mcp_protocol}")
        
    except Exception as e:
        print(f"❌ Configuration error: {e}")
else:
    print("⏭️ Skipping configuration tests - ConfigManager not available")

Testing configuration management...
✅ Config manager created
✅ Configuration saved
✅ Configuration loaded: {'default_port': 7860, 'auto_reload': True, 'mcp_protocol': 'auto', 'log_level': 'INFO'}
Default port: 7860
Auto reload: True
MCP protocol: auto


## 🎯 Template Testing

In [19]:
# Test different templates
templates_to_test = ["calculator", "image-generator"]

for template in templates_to_test:
    server_name = f"test-{template}-server"
    print(f"\nTesting {template} template...")
    
    result = run_cli_command(["server", "create", server_name, "--template", template])
    
    if result['success']:
        print(f"✅ {template} server created")
        
        # Check created files
        server_path = Path(server_name)
        if server_path.exists():
            files = list(server_path.glob("*"))
            print(f"📄 Files: {[f.name for f in files]}")
            
            # Check app.py content
            app_file = server_path / "app.py"
            if app_file.exists():
                content = app_file.read_text()
                print(f"📝 App.py length: {len(content)} characters")
                print(f"🔍 Contains 'gradio': {'gradio' in content.lower()}")
                print(f"🔍 Contains template keyword: {template.replace('-', '_') in content.lower()}")
            else:
                print("❌ app.py not found")
        else:
            print(f"❌ Server directory {server_name} not found")
    else:
        print(f"❌ {template} server creation failed: {result.get('stderr', 'Unknown error')}")


Testing calculator template...
✅ calculator server created
📄 Files: ['app.py', 'README.md', 'requirements.txt']
📝 App.py length: 2070 characters
🔍 Contains 'gradio': True
🔍 Contains template keyword: True

Testing image-generator template...
✅ image-generator server created
📄 Files: ['app.py', 'requirements.txt']
📝 App.py length: 3814 characters
🔍 Contains 'gradio': True
🔍 Contains template keyword: False


## 🚀 Performance Testing

In [20]:
# Test performance of various operations
import time

print("Testing performance...")

if all_imports_successful and 'MCPServer' in globals():
    # Test MCP server creation time
    start_time = time.time()
    test_server = MCPServer("perf-test", "1.0.0")

    # Add multiple tools
    for i in range(10):
        @test_server.tool(f"tool_{i}", f"Test tool {i}")
        async def test_func(x: str) -> str:
            return f"Result: {x}"

    creation_time = time.time() - start_time
    print(f"⏱️ MCP server creation (10 tools): {creation_time:.3f}s")

    # Test Gradio conversion time
    start_time = time.time()
    gradio_funcs = test_server.to_gradio_functions()
    conversion_time = time.time() - start_time
    print(f"⏱️ Gradio conversion: {conversion_time:.3f}s")

    # Test tool execution time
    start_time = time.time()
    for _ in range(10):
        await test_server.tools[0].handler("test")
    execution_time = time.time() - start_time
    print(f"⏱️ 10 tool executions: {execution_time:.3f}s")

    print("✅ Performance testing completed")
else:
    print("⏭️ Skipping performance tests - MCP dependencies not available")

Testing performance...
⏱️ MCP server creation (10 tools): 0.010s
⏱️ Gradio conversion: 0.003s
⏱️ 10 tool executions: 0.000s
✅ Performance testing completed


## 🔍 Error Handling Testing

In [21]:
# Test error handling for CLI commands
print("🧪 Testing error handling...")

# Test commands that should fail
test_cases = [
    (["invalid-command"], "Invalid command"),
    (["server", "create"], "Missing server name"),
    (["client", "connect", "invalid-url"], "Invalid URL")
]

failed_as_expected = 0
for cmd, description in test_cases:
    result = run_cli_command(cmd)
    expected_failure = not result['success']
    if expected_failure:
        failed_as_expected += 1
        print(f"✅ {description}: Failed as expected")
    else:
        print(f"⚠️ {description}: Should have failed but succeeded")

# Test commands that should succeed but return informative messages
info_cases = [
    (["server", "create", "test", "--template", "nonexistent"], "Nonexistent template"),
    (["server", "info", "nonexistent-server"], "Nonexistent server")
]

for cmd, description in info_cases:
    result = run_cli_command(cmd)
    if result['success'] and ('not found' in result['stdout'].lower() or 'template' in result['stdout'].lower()):
        print(f"✅ {description}: Returned helpful error message")
    else:
        print(f"❌ {description}: Unexpected behavior")

print(f"✅ Error handling tests completed ({failed_as_expected}/{len(test_cases)} failed as expected)")

🧪 Testing error handling...
✅ Invalid command: Failed as expected
✅ Missing server name: Failed as expected
✅ Invalid URL: Failed as expected
✅ Nonexistent template: Returned helpful error message
✅ Nonexistent server: Returned helpful error message
✅ Error handling tests completed (3/3 failed as expected)


In [22]:
# Test MCP tool error handling
print("Testing MCP tool error handling...")

if all_imports_successful and 'MCPServer' in globals():
    error_server = MCPServer("error-test", "1.0.0")

    @error_server.tool("failing_tool", "A tool that can fail")
    async def failing_tool(input_val: str) -> str:
        if input_val == "fail":
            raise ValueError("This is a test failure")
        return f"Success: {input_val}"

    # Test successful execution
    try:
        result = await error_server.tools[0].handler("success")
        print(f"✅ Successful execution: {result}")
    except Exception as e:
        print(f"❌ Unexpected error in successful case: {e}")

    # Test error case
    try:
        result = await error_server.tools[0].handler("fail")
        print(f"❌ Should have failed but got: {result}")
    except ValueError as e:
        print(f"✅ Error handled correctly: {e}")
    except Exception as e:
        print(f"❌ Unexpected error type: {type(e)} - {e}")
else:
    print("⏭️ Skipping MCP error handling tests - dependencies not available")

Testing MCP tool error handling...
✅ Successful execution: Success: success
✅ Error handled correctly: This is a test failure


## 📊 Test Results Summary

In [23]:
# Cleanup and summary
os.chdir(original_cwd)

print("\n" + "="*50)
print("🎯 MANUAL TESTING SUMMARY")
print("="*50)

print("\n✅ Completed Tests:")
print("  🔧 Basic CLI functionality")
print("  🖥️ Server management")
print("  🔌 MCP server functionality")
print("  🌐 Web dashboard creation")
print("  📦 Registry operations")
print("  🔧 Configuration management")
print("  🎯 Template testing")
print("  🚀 Performance testing")
print("  🔍 Error handling")

print("\n📋 Next Steps:")
print("  1. Run automated test scripts: python tests/test_cli_comprehensive.py")
print("  2. Run MCP functionality tests: python tests/test_mcp_functionality.py")
print("  3. Test dashboard manually: gmp dashboard")
print("  4. Test real server deployment")
print("  5. Review QA/QC checklist: QA_QC_CHECKLIST.md")

print("\n💡 Tips:")
print("  - Use 'gmp --help' for complete CLI reference")
print("  - Check test_environment/ for created test servers")
print("  - Run individual cells to debug specific issues")
print("  - Monitor logs when testing server startup")

print(f"\n📁 Test artifacts created in: {test_dir.absolute()}")
if test_dir.exists():
    artifacts = list(test_dir.glob("*"))
    print(f"📄 Artifacts: {[a.name for a in artifacts]}")

print("\n🎉 Manual testing notebook completed!")


🎯 MANUAL TESTING SUMMARY

✅ Completed Tests:
  🔧 Basic CLI functionality
  🖥️ Server management
  🔌 MCP server functionality
  🌐 Web dashboard creation
  📦 Registry operations
  🔧 Configuration management
  🎯 Template testing
  🚀 Performance testing
  🔍 Error handling

📋 Next Steps:
  1. Run automated test scripts: python tests/test_cli_comprehensive.py
  2. Run MCP functionality tests: python tests/test_mcp_functionality.py
  3. Test dashboard manually: gmp dashboard
  4. Test real server deployment
  5. Review QA/QC checklist: QA_QC_CHECKLIST.md

💡 Tips:
  - Use 'gmp --help' for complete CLI reference
  - Check test_environment/ for created test servers
  - Run individual cells to debug specific issues
  - Monitor logs when testing server startup

📁 Test artifacts created in: c:\Users\seanp\gradio-mcp-playground\tests\test_environment
📄 Artifacts: ['test-basic-server', 'test-calculator-server', 'test-image-generator-server']

🎉 Manual testing notebook completed!


## 🧹 Cleanup (Optional)

In [24]:
# Uncomment to clean up test environment
# import shutil
# if test_dir.exists():
#     shutil.rmtree(test_dir)
#     print(f"🧹 Cleaned up test directory: {test_dir}")
# else:
#     print("✨ Test directory already clean")

print("💡 Uncomment the code above to clean up test artifacts")

🧹 Cleaned up test directory: test_environment
💡 Uncomment the code above to clean up test artifacts
